 <font size="6">Нейронные сети</font>

#  Ограничения Линейного классификатора

Вспомним материал лекции №2

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-11.jpg" width="550">




- Линейный классификатор: скалярное произведение
- Лосс - функции: SVM,CrossEntropyLoss
- Градиентный спуск
- Оценка точности Линейного классификатора (0.38)

Попробуем визуализировать, шаблоны, получающиеся усреднением изображений. 

In [ ]:
from IPython.display import clear_output
!wget http://edunet.kea.su/repo/src/L05_Neural_networks/lc_cifar10_weights.txt
clear_output()

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

# Display templates 
plt.rcParams["figure.figsize"] = (25, 10)

W = torch.from_numpy(np.loadtxt("lc_cifar10_weights.txt")) # 3073x10
print(W.shape)

# Remove bias
W = W[:-1, :]
print(W.shape)

# Denormalize
w_min = torch.min(W)
w_max = torch.max(W)
templates =  255*(W-w_min)/(w_max-w_min)

# Display templates
labels = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
    plt.subplot(1, 10, i+1)
    img = templates[:,i].view(3, 32, 32).permute(1, 2, 0).type(torch.uint8)
    plt.imshow(img)
    plt.axis('off')
    plt.title(labels[i])

Ограничение этого подхода состоит в том, что покольку для каждого класса существует один шаблон, он будет сохранять информацию обо всех объектах класса сразу (например, на получившихся шаблонах у лошади две головы, машина красная и т.д.). Такой подход не допускает вариативности, а это будет мешать модели запоминать разные варианты объектов одного класса, что ограничивает точность модели.

##  ХОR — проблема

У линейного классификатора есть существенные ограничения применения. Рассмотрим задачу XOR. На вход подаётся упорядоченный набор из двух чисел согласно таблице истинности xor. Задача линейного классификатора сопоставить этим числам их класс согласно таблице. Графически два входных числа можно изобразить на плоскости как координаты и цветом обозначить их истинный класс. Задача классификатора - построить линию, отделяющую красные точки (класс 0) от зелёных точек (класс 1). Однако, видно, что одной линией это сделать геометрически невозможно.

То есть линейный классификатор уже не может справится с этой задачей.

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/1.png"  width="600">

## Проблемы классификации более сложных объектов

Когда компьютер смотрит на изображение, то не видит целостное представление кошки или любого другого объекта. Он видит лишь гигантскую сетку чисел. Например, если размер изображения 800 на 600 и каждый пиксель представлен тремя числами для красного, зелёного и синего каналов, то получится сетка из 800х600х3 = 1,440,000 чисел. 

Например, если мы снимем кошку с другого ракурса или при ином освещении, то вся сетка чисел будет выглядеть совершенно иначе. Помимо этого, животные могут принимать множество различных поз, или же на фотографии может оказаться только часть кошки, например, хвост. Алгоритмы распознавания должны быть устойчивы к таким изменениям. Эта проблема получила название «семантический разрыв» — непонимание информации, которая заключена в данных.

Вот лишь малая часть параметров, от которых зависит точность распознавания линейного классификатора.

Обратите внимание, что на всех изображениях, в том или ином виде присутствуют кошки. При решении задачи классификации, например "кошка/собака", все изображения должны быть определены как "кошка".



<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/2.png"  width="700">

Помимо этих сложностей, существует ещё проблема внутриклассовых вариаций, когда одно понятие охватывает множество визуальных проявлений. Например, кошки могут быть разных пород, возрастов и размеров. И методы распознавания должны обрабатывать все возможные варианты.

Один из подходов к решению этой проблемы &mdash; модифицировать модель таким образом, чтобы на выходе у нее было не 10, а 100 шаблонов, позволяющих запоминать разные объекты одного класса и далее использовать эти шаблоны для разбиения объектов на классы.

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/8.png"  width="700">

Реализуем эту модель на основе линейного классификатора из лекции №2:

In [ ]:
def predict(self, x): # x is image
  x = torch.cat((x, torch.Tensor([1])), dim=0) # bias trick
  scores = x.matmul(self.W) # dot product
  return torch.argmax(scores)

Применяем к выходам классификатора еще один классификатор. Будет ли работать данная модель?

In [ ]:
x = torch.rand(3072)
W1 = torch.randn(3072, 100)*0.0001 # without bias
W2 = torch.randn(100, 10)*0.0001 # without bias
scores1 = x.matmul(W1) # matrix multiplication, equivalent x@W1
scores2 = scores1.matmul(W2)

print(scores2)

Нетрудно заметить, что последовательное применение двух классификаторов ко входным данным эквивалентно применению одного классификатора с матрицей весов, равной произведению двух матриц весов классификаторов примененных последовательно.

$$scores2 = x\cdot W1\cdot W2$$ 
$$W = W1\cdot W2$$
$$scores2 = x\cdot W$$ 

Для того, чтобы последовательно примененные классификаторы не вырождались в один, необходимо применить нелинейность к их выходам, например, сделаем так, чтобы каждый шаблон, предсказывающий класс объекта воспринимал только те сигналы:

In [ ]:
scores1 = x.matmul(W1) 
print(f"\nScores1 {scores1}")
activations = torch.maximum(torch.tensor(0), scores1) # Use only patterns with big score
print(f"\nActivations {activations}" )
scores2 = activations.matmul(W2)
print(f"\nScores2 {scores2}")

Нелинейность:

<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-09.jpg" width="450">

Такая конструкция называется **функцией активации**. И мы уже пользовались подобной когда разбирали Cross entropy loss (Softmax)

Приведем код в порядок:

In [ ]:
class NeuralNet:
  def __init__(self):
    self.W1 = torch.randn(3072, 100)*0.0001 # without bias
    self.W2 = torch.randn(100, 10)*0.0001 # without bias

  def predict(self,x):
    scores1 = x.matmul(self.W1) # Linear / Fully connected layer1
    activations1 = torch.maximum(torch.tensor(0), scores1) # ReLU activation
    scores2 = activations1.matmul(self.W2)
    return activations1, scores2

x = torch.rand(3072) # input image
nn = NeuralNet()
activations, scores = nn.predict(x)
print(f'activations {activations}\nscores: {scores}')

#  Переход от ЛК к прецептрону

Ядром этой операции является скалярное произведение. 



<img src ="http://edunet.kea.su/repo/src/L04_Feature_Engineering/img/L04_skalyar.png" width="750">

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/9.png"  width="550">


И она соответствует одиному слою искусственной нейроннной сети (за исключением функции активации).



<img src ="http://edunet.kea.su/repo/src/L02_Linear_classifier/img/L02_Linear_classifier-10.jpg" width="800">

##  Перцептрон - нейросеть с одним скрытым слоем

В 1957 году Фрэнк Розенблатт изобрёл вычислительную систему «Марк-1», которая стала первой реализацией перцептрона. Этот алгоритм тоже использует интерпретацию линейного классификатора и функцию потерь, но на выходе выдаёт либо 0, либо 1, без промежуточных значений.

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img/Rosenblatt.jpg"  width="300">

[Перцептрон](https://ru.wikipedia.org/wiki/%D0%9F%D0%B5%D1%80%D1%86%D0%B5%D0%BF%D1%82%D1%80%D0%BE%D0%BD)

В 1960 году Бернард Уидроу и Тед Хофф разработали однослойную нейросеть ADALINE и её улучшенную версию — трёхслойную MADALINE. Это были первые глубокие (для того времени) архитектуры, но в них ещё не использовался метод обратного распространения ошибки. 

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img/percepton.PNG"  width="500">

## Принцип работы перцептрона

В основу идеи перцептрона были положены биологические процессы, например светочувствительные клетки сетчатки глаза. Каждый рецептор может находиться в одном из двух состояний — покоя или возбуждения.

Перцептрон состоит из трёх типов элементов:

* Простым S-элементом (сенсорным) является чувствительный элемент, который от воздействия какого-либо из видов энергии (например, света, звука, давления, тепла и т. п.) вырабатывает сигнал. Если входной сигнал превышает некоторый порог θ, на выходе элемента получаем +1, в противном случае — 0.

* Простым A-элементом (ассоциативным) называется логический решающий элемент, который даёт выходной сигнал +1, когда алгебраическая сумма его входных сигналов превышает некоторую пороговую величину θ (говорят, что элемент активный), в противном случае выход равен нулю.

* Простым R-элементом (реагирующим, то есть действующим) называется элемент, который выдаёт сигнал +1, если сумма его входных сигналов является строго положительной, и сигнал −1, если сумма его входных сигналов является строго отрицательной. Если сумма входных сигналов равна нулю, выход считается либо равным нулю, либо неопределённым.

В данном случае **скрытым слоем** называются А-элементы.

А **функцией активации** для А-элементов является пороговая функция:

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img/L05_1-1.png"  width="300">

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/4.png"  width="300">

Для R-элементов же функция активации (signum) выглядит следующим образом:

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/3.png"  width="300">

Таким образом поступающие от датчиков сигналы передаются ассоциативным элементам, а затем реагирующим элементам. Перцептроны позволяют создать набор «ассоциаций» между входными стимулами и необходимой реакцией на выходе. В биологическом плане это соответствует преобразованию, например, зрительной информации в физиологический ответ от двигательных нейронов. Согласно современной терминологии, перцептроны могут быть классифицированы как искусственные нейронные сети:


* с одним скрытым слоем
* с пороговой передаточной функцией
* с прямым распространением сигнала

Чтобы «научить» перцептрон классифицировать образы, был разработан специальный итерационный метод обучения проб и ошибок, напоминающий процесс обучения человека — метод коррекции ошибки.

##  Обучение перцептрона

Нам мало знать, как распространяется сигнал в перцептроне и какие функции используются при прямом распространении. Важно ещё уметь **обучить** перцептрон - то есть подстроить веса и пороги таким образом, чтобы наша нейронная сеть могла решать задачу.

**Метод коррекции ошибки** — метод обучения перцептрона, предложенный Фрэнком Розенблаттом. Представляет собой такой метод обучения, при котором вес связи не изменяется до тех пор, пока текущая реакция перцептрона остается правильной. При появлении неправильной реакции вес изменяется на единицу, а знак (+/-) определяется противоположным от знака ошибки.

##  Принципиальные ограничения перцептрона

Первые успехи исследований перцептронов и других нейросетей вызвал взрыв активности и энтузиазма. В течение некоторого времени казалось, что ключ к интеллекту найден, и воспроизведение человеческого мозга является лишь вопросом конструирования достаточно большой сети.

Однако уже через несколько лет было доказано, что перцептрон не может воспроизвести ряд простых функций, например функцию xor. После этого открытия интерес к нейросетям резко падал и они находились в забвении. По сути, даже опубликовать статью по данной теме было затруднительно.

Возможности перцептронов оказались довольно ограниченными.

* Спектр решаемых задач: с помощью перцептрона можно решать задачи классификации и апроксимации. С большими ограничениями (например классификация возможна только бинарная)
* Выбор непрерывной функции активации не влияет на достижение решения. Единственное, для чего имеет смысл усложнять функцию активации по сравнению с пороговой (которая является самой наипростейшей), — это возможность интерпретации выходов нейронов как вероятностей принадлежности к соответствующему классу, что в свою очередь может повлиять на качество прогноза.
* Нет способа (на тот момент) обучать многослойные перцептроны.
* Перцептрон основывается на статистическом обучении, то для него доступны те задачи, в которых объекты каждого класса имеют общие фрагменты, но могут быть в разных комбинациях, например, задачи распознавания образов.
*  1969 году Марвин Минский и Сеймур Паперт посвятили критике перцептрона целую книгу. Минский описывал специальные задачи такие как «чётность» и «один в блоке», которые показывают ограничения перцептрона в том, что он не может распознавать инвариантные входные данные (изображения) бесконечного порядка. А в частности, при распознавании чётности конечного порядка первый слой перцептрона вынужден становиться полно связным.

Типичные задачи, с которыми не может справится перцептрон:

1, 2 — преобразования группы переносов

3 — из какого количества частей состоит фигура?

4 — внутри какого объекта нет другой фигуры?

5 — какая фигура внутри объектов повторяется два раза?

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/5.png"  width="450">

#  Многослойные сети

По мере развития мощности компьютеров, теоретической базы, появления больших датасетов и метода обратного распространения ошибки, появилась возможность строить более сложные сети - многослойные нейронные сети (многослойные перцептроны) или же в современном понимании просто **нейронные сети**.

Пример **полносвязной (fully connected network)** нейронной сети с двумя скрытыми слоями:

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img/nn_fully_connected.png"  width="400">

[Neural machine translation](https://arxiv.org/pdf/1409.0473.pdf)

##  Функции потерь (loss functions)

Положим у нас есть нейронная сеть с некоторыми весами, прежде всего мы должны понять насколько она точна - то есть наши ожидания соответствуют результату работы нейронной сети. Мы подали на вход нейронной сети изображение, сигналы прошли через наши слои и функции активации вперёд **(forward propagation)**, и на выходе мы имеем некоторый ответ. Как его оценить? Насколько он точен?

Для оценки соответствия полученного результата ожидаемому используют функции потерь. С помощью функции потери оценивают ошибку нейронной сети. 

Функция потерь в нейронной сети должна удовлетворять двум условиям:

* Функция потерь должна быть записана как среднее;
* Функция потерь не должна зависеть от каких-либо активационных значений нейронной сети, кроме значений, выдаваемых на выходе.

[PyTorch Docs](https://pytorch.org/docs/stable/nn.html#loss-functions)

###  Mean squared error

Mean Squared Error (MSE) - это средняя квадратическая ошибка. Данная функция потерь очень популярная, поскольку она проста для понимания и реализации, и в целом работает довольно хорошо. Чтобы рассчитать MSE, вы берете разницу между предсказаниями вашей модели и фактическими значениями, вычитаете их, возводя разницу в квадрат и затем усредняете по всему набору данных.
Результат всегда положительный, независимо от знака предсказанных и истинных значений, и идеальное значение равно 0,0.

$$MSE=\frac{1}{n}\sum_{i=1}^n(Y_i - \hat{Y_i})^2$$

```python
def MSE(y_predicted, y_actual):    
    squared_error = (y_predicted-y_actual)**2
    sum_squared_error = np.sum(squared_error)
    mse = sum_squared_error/y_actual.size
    return mse
```

```python
torch.nn.MSELoss()
```

In [ ]:
import torch
import torch.nn as nn

loss = nn.MSELoss()
input = torch.Tensor([0.5, -0.25, 0.75])
print(f'input: {input}')
target = torch.Tensor([1, 0.25, 0.25])
print(f'target: {target}')
output = loss(input, target)
print(f'output: {output}')

* **Преимущество:** MSE отлично подходит для проверки того, что наша обученная модель не имеет выбросов с огромными ошибками, поскольку MSE придает большее значение этим ошибкам из-за квадрата функции.


* **Недостаток:** Если наша модель делает одно очень плохое предсказание, то квадратичная часть функции увеличивает ошибку. Тем не менее, во многих практических случаях мы не очень заботимся об этих выбросах и стремимся к более всесторонней модели, которая достаточно хороша для большинства случаев.

###  Mean Absolute Error 

Средняя абсолютная ошибка (MAE) лишь немного отличается по определению от MSE, но, что интересно, обеспечивает почти совершенно противоположные свойства. Чтобы рассчитать MAE, вы берете разницу между предсказаниями вашей модели и основополагающей правдой, применяете абсолютное значение к этой разнице, а затем усредняете его по всему набору данных.

$$MAE=\frac{1}{n}\sum_{i=1}^n|Y_i - \hat{Y_i}|$$

```python
def MAE(y_predicted, y_actual):
    abs_error = np.abs(y_predicted - y_actual)
    sum_abs_error = np.sum(abs_error)
    mae = sum_abs_error / y_actual.size
    return mae
```

```python
torch.nn.L1Loss
```

In [ ]:
loss = nn.L1Loss()
input = torch.Tensor([0.5, -0.25, 0.75])
print(f'input: {input}')
target = torch.Tensor([1, 0.25, 0.25])
print(f'target: {target}')
output = loss(input, target)
print(f'output: {output}')

* **Преимущество:** Прелесть MAE заключается в том, что её преимущество напрямую покрывает недостаток MSE. Поскольку мы берем абсолютное значение, все ошибки будут взвешены в одной линейной шкале. Таким образом, в отличие от MSE, мы не будем придавать слишком большой вес нашим выбросам, а наша функция потерь обеспечивает общую и даже меру того, насколько хорошо работает наша модель.


* **Недостаток:** Если мы действительно заботимся о прогнозируемых отклонениях нашей модели, то MAE не будет столь же эффективным. Большие ошибки, возникающие из-за выбросов, в конечном итоге взвешиваются точно так же, как и более маленькие ошибки. Это может привести к тому, что наша модель в большинстве случаев будет отличной, но время от времени будет делать несколько очень плохих прогнозов.

###  Cross entropy

Кроссэнтропия (перекрёстная энтропия) - одна из самых важных функций, используемых в обучении нейронных сетей.

Пусть:
* **p** - истинное распределение
* **q** - прогнозируемое распределение

Перекрёстная энтропия между двумя распределениями вероятностей **p** и **q** измеряет среднее число бит, необходимых для опознания события из набора возможностей, если используемая схема кодирования базируется на заданном распределении вероятностей **q**, вместо «истинного» распределения **p**.

$$ H(p,q) = -E_p[\log q]$$

Где:
* Hp - оператор математического ожидания относительно распределения

Однако чаще кроссэнтропию определяют с помощью энтропии и расстояния Кульбака-Лейблера:

$$H(p,q) = H(p) +D_{KL}(p||q)$$

В случае нейронных сетей, где вероятности представлены дискретными выходами, формула превращается в:


$$H(p,q)=-\sum_xp(x)\log q(x)$$

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/10.png"  width="800">



Поскольку чаще всего кроссэнтропию используют после softmax то в готовых реализациях softmax объединяют с кроссэнтроией

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/11.png"  width="700">

```python
torch.nn.CrossEntropyLoss
```

In [ ]:
loss = nn.CrossEntropyLoss()
input = torch.rand(3, 5)
print(f'input: {input}')
target = torch.empty(3, dtype=torch.long).random_(5)
print(f'target: {target}')
output = loss(input, target)
print(f'output: {output}')

* **Преимущества:** Важное свойство кроссэнтропии - возможность работать с весами для классов. А значит и возможность применения этой функции потерь при работе с несбалансированным датасетом.
* **Недостатки:** Вычислительная сложность выше чем MSE или MAE

###  Binary cross entropy

В случае когда количество классов равно двум функция кроссэнтропии определяется как:

$$H_p(q)=-\frac{1}{N}\sum_{i=1}^N y_i\cdot log(p(y_i))+(1-y_i)\cdot log(1-p(y_i))$$

Бинарная кроссэнтропия. В отличие от кроссэнтропии она не зависит от каждого компонента вектора (класса), что означает, что на потери, вычисленные для каждого компонента вектора вывода CNN, не влияют значения других компонентов. Вот почему он используется для классификации с несколькими метками, когда понимание элемента, принадлежащего определенному классу, не должно влиять на решение для другого класса.


        torch.nn.BCELoss


In [ ]:
torch.__version__

In [ ]:
loss = nn.BCELoss()
input = torch.rand(3)
print(f'input: {input}')
target = torch.empty(3, dtype=torch.float).random_(2)
print(f'target: {target}')
output = loss(input, target)
print(f'output: {output}')

###  Итоги

Кросс-энтропия предпочтительнее для *классификации*, в то время как среднеквадратичная ошибка является одним из лучших вариантов для *регрессии*. Это происходит непосредственно из самой постановки задач &mdash; в классификации вы работаете с очень конкретным набором возможных выходных значений, поэтому MSE плохо определен (поскольку он не обладает такого рода знаниями, поэтому наказывает ошибки несовместимым образом). Чтобы лучше понять явления, полезно проследить и понять отношения между ними.

И то, и другое можно рассматривать как оценки максимального правдоподобия, просто с различными предположениями о зависимой переменной.

##  Функции активации

Каждый элемент нейронной сети (нейрон) имеет один или несколько входов и один выход. Нейрон представляет собой систему из двух элементов — сумматора и функции активации. 

Рассмотрим нейрон, у которого взвешенная сумма входов:

$$ z=\sum_{i=1}^nw_i \cdot x_i+b=WX+b$$

где $w_i$ и $x_i$ &mdash;   вес и входное значение $i$-го входа, $W$ и $X$ &mdash; векторы весов и входов, а $b$ &mdash; смещение. $z$ может принимать любые значения в диапазоне $(-\infty;+\infty)$, оно  передается в функцию активации $\sigma()$, которая определяет выходное значение этого нейрона: $$a=\sigma(WX+b)=\sigma(z)$$

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/l5_out_1.png"  width="900">

В математическом смысле функция активации преобразует результат работы нейрона в известный диапазон значений, например $(0;1)$.

Историческим примером функции активации является пороговая функция активации, вдохновленная активацией нейронов, использовавшаяся в перцептронах &mdash; первых нейронных сетях.

В биологических нейронных сетях функция активации определяется пороговым потенциалом, при достижении которого происходит возбуждение потенциала действия в клетке. В наиболее простой форме эта функция является двоичной — то есть нейрон либо возбуждается, либо нет. 

Таким же образом ведет себя пороговая функция активации, использовавшаяся в перцептронах &mdash; первых нейронных сетях:

$$f(x) =
\begin{cases}
0, &\text{$x<b$} \\ 
1, &\text{$x\geq b$}
\end{cases}
$$

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/4.png"  width="300">

Производная пороговой функции активации:

$$f'(x) =
\begin{cases}
0, &\text{$x\neq b$} \\ 
DNE, &\text{$x= b$}
\end{cases}
$$

Главным недостатком пороговой функции активации является то, что поскольку производная пороговой функции неопределена при $x=b$, а во всех остальных случаях равна 0, не может быть использована для оптимизации параметров нейронной сети методом градиентного спуска, использующимися при обучении современных нейронных сетей. 

В настоящее время пороговая функция активации не используется, поскольку не удовлетворяет требованиям, которые предъявляются к современным нейронным сетям

###  Свойства функций активации

Функции активации должны обладать следующими свойствами:

* **Нелинейность:**
Функция активации необходима для введения нелинейности в нейронные сети. Если функция активации не применяется, выходной сигнал становится простой линейной функцией. Неактивированная нейронная сеть будет действовать как линейная регрессия с ограниченной способностью к обучению:
$$\hat{y}=NN(X,W_1,...,W_n)=X\cdot W_1\cdot ...\cdot W_n=X\cdot W$$ 
Только нелинейные функции активации позволяют нейронным сетям решать задачи аппроксимации нелинейных функций:
$$\hat{y}=NN(X,W_1,...,W_n)=\sigma(...\sigma(X\cdot W_1)...\cdot W_n)\neq X\cdot W$$

* **Возможность прохождения градиента:** 
Функции активации должны быть способными пропускать градиент, чтобы было возможно оптимизировать параметры сети градиентными методами, в частности использовать алгоритм обратного распространения ошибки.

###  Типы функций активации

Рассмотрим наиболее популярные функции активации и обсудим их преимущества и недостатки.

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img/L05_25-1.png"  width="700">

[AI in Pursuit of Happiness, Finding
Only Sadness: Multi-Modal Facial
Emotion Recognition Challenge  ](https://arxiv.org/pdf/1911.05187.pdf)

####  **Логистическая (сигмоидальная)**

Sigmoid (сигмоидальная) для одномерного случая - используется в задачах классификации, в основном после выхода последнего нейрона. Позволяет определить вероятность принадлежности к одному из двух классов (0 или 1)

$$f(x)=\frac{1}{1+e^{-x}}=\frac{e^{x}}{e^{x}+1}=\frac{1}{2}+\frac{1}{2}tanh(\frac{x}{2})$$

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img/logistic_plot.png"  width="500">

Производная логистической функции:

$$\frac{d}{dx}f(x)=\frac{e^x\cdot (1+e^x)-e^x \cdot e^x}{(1+e^x)^2}=\frac{e^x}{(1+e^x)^2}=f(x)(1-f(x))=f(x)f(-x)$$

Если активационная функция не бинарная (не как пороговая), то для нейрона возможны значения “активирован на 50%”, “активирован на 20%” и так далее. Если активированы несколько нейронов, можно найти нейрон с наибольшим значением активационной функции.

Так как существуют промежуточные значения на выходе нейрона, **процесс обучения проходит более гладко и быстро**, а вероятность появления нескольких полностью активированных нейронов во время тренировки снижается по сравнению со ступенчатой функцией активации (хотя это зависит от того, что вы обучаете и на каких данных).

```python
torch.nn.Sigmoid()
```

In [ ]:
activation = nn.Sigmoid()
input = torch.randn(5)*5
output = activation(input)
print(f'input: {input}\noutput: {output}')

Сигмоида выглядит гладкой и подобна пороговой функции.

**Достоинства:**

Во-первых, сигмоида — нелинейна по своей природе, а комбинация таких функций производит тоже нелинейную функцию. Поэтому мы можем конструировать многослойные сети.

Еще одно достоинство такой функции — она гладкая, что делает выходной сигнал аналоговым, в отличие от ступенчатой функции. Для сигмоиды также характерен гладкий градиент.


**Недостатки:**

Насыщение сигмоиды приводит к затуханию градиентов. Крайне нежелательное свойство сигмоиды заключается в том, что при насыщении функции с той или иной стороны (0 или 1), градиент на этих участках становится близок к нулю. Напомним, что в процессе обратного распространения ошибки данный (локальный) градиент умножается на общий градиент. Следовательно, если локальный градиент очень мал, он фактически обнуляет общий градиент. В результате, сигнал почти не будет проходить через нейрон к его весам и рекурсивно к его данным. Кроме того, следует быть очень осторожным при инициализации весов сигмоидных нейронов, чтобы предотвратить насыщение. Например, если исходные веса имеют слишком большие значения, большинство нейронов перейдет в состояние насыщения, в результате чего сеть будет плохо обучаться.

Выход сигмоиды не центрирован относительно нуля. Это свойство является нежелательным, поскольку нейроны в последующих слоях будут получать значения, которые не центрированы относительно нуля, что оказывает влияние на динамику градиентного спуска. Если значения, поступающие в нейрон, всегда положительны (например, $x > 0$ поэлементно в $f = wx + b$), тогда в процессе обратного распространения ошибки все градиенты весов $w$ будут либо положительны, либо отрицательны (в зависимости от градиента всего выражения $f$). Это может привести к нежелательной зигзагообразной динамике обновлений весов. Однако следует отметить, что когда эти градиенты суммируются по пакету, итоговое обновление весов может иметь различные знаки, что отчасти нивелирует описанный недостаток. Таким образом, отсутствие центрирования является неудобством, но имеет менее серьезные последствия, по сравнению с проблемой насыщения.

####  **Гиперболический тангенс**

Гиперболический тангенс очень похож на сигмоиду. И действительно, это скорректированная сигмоидная функция.

$$f(x)=tanh(x)=\frac{2}{1+e^{-2x}}-1$$

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/0-447458-798873.png"  width="500">

Гиперболический тангенс используется в рекуррентных нейронных сетях, поскольку может принимать отрицательные значения, что позволяет как увеличивать, так и уменьшать скрытое состояние ячейки памяти (подробнее в будущих лекциях).

**Достоинства**: Гиперболический тангенс имеет те же характеристики, что и у сигмоиды, рассмотренной ранее. Его природа нелинейна, он хорошо подходит для комбинации слоёв. В отличие от логистической функции (для которой значения функции центрированы около 0.5), диапазон значений функции $(-1, 1)$, таким образом, её значения центрированы относительно 0, что позволяет нивелировать соответствующие недостатки сигмоиды. Стоит отметить, что градиент тангенциальной функции больше, чем у сигмоиды (производная круче). Решение о том, выбрать ли сигмоиду или тангенс, зависит от ваших требований к амплитуде градиента. 

**Недостатки**: Также как и сигмоиде, гиперболическому тангенсу свойственная проблема исчезновения градиента в области насыщения функции.


```python
torch.nn.Tanh()
```

In [ ]:
activation = nn.Tanh()
input = torch.randn(5)*5
output = activation(input)
print(f'input: {input}\noutput: {output}')

####  **ReLu**

В последние годы большую популярность приобрела функция активации под названием «выпрямитель» (rectifier, по аналогии с однополупериодным выпрямителем в электротехнике). Нейроны с данной функцией активации называются ReLU (rectified linear unit). ReLU имеет следующую формулу и реализует простой пороговый переход в нуле:



$$relu(x)=max(0,x)$$

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/l5_out_3.png"  width="500">

Производная ReLu:

$$\frac{d}{dx}relu(x) =
\begin{cases}
\frac{d}{dx}0, &\text{$x<0$} \\ 
\frac{d}{dx}x, &\text{$x\geq0$}
\end{cases}=
\begin{cases}
0, &\text{$x<0$} \\ 
1, &\text{$x\geq0$}
\end{cases}
$$

```python
torch.nn.ReLU()
```

In [ ]:
activation = nn.ReLU()
input = torch.randn(5)
output = activation(input)
print(f'input: {input}\noutput: {output}')

Рассмотрим положительные и отрицательные стороны ReLU.

**Достоинства:**

Вычисление сигмоиды и гиперболического тангенса требует выполнения ресурсоемких операций, таких как возведение в степень, в то время как ReLU может быть реализован с помощью простого порогового преобразования матрицы активаций в нуле. Кроме того, ReLU не подвержен насыщению.
Применение ReLU существенно повышает скорость сходимости стохастического градиентного спуска (в некоторых случаях до 6 раз) по сравнению с сигмоидой и гиперболическим тангенсом. Считается, что это обусловлено линейным характером и отсутствием насыщения данной функции.



**Недостатки:**

К сожалению, ReLU не всегда достаточно надежны и в процессе обучения могут выходить из строя («умирать»). Например, большой градиент, проходящий через ReLU, может привести к такому обновлению весов, что данный нейрон никогда больше не активируется. Если это произойдет, то, начиная с данного момента, градиент, проходящий через этот нейрон, всегда будет равен нулю. Соответственно, данный нейрон будет необратимо выведен из строя. Например, при слишком большой скорости обучения (learning rate), может оказаться, что до 40% ReLU «мертвы» (то есть, никогда не активируются). Эта проблема решается посредством выбора надлежащей скорости обучения.

#### **Leaky ReLu**

ReLU с «утечкой» (leaky ReLU, LReLU) представляет собой одну из попыток решить описанную выше проблему выхода из строя обычных ReLU. Обычный ReLU на интервале $x < 0$ дает на выходе ноль, в то время как LReLU имеет на этом интервале небольшое отрицательное значение (угловой коэффициент около 0,01). То есть функция для LReLU имеет вид  $f(x) = \alpha x$ при $x < 0$ и $f(x) = x$ при $x ≥ 0$, где $\alpha$ – малая константа. 

$$lrelu(x)=max(0.01x,x)$$

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img/lrelu_plot.png"  width="400">

Производная leaky ReLU:

$$\frac{d}{dx}lrelu(x)=\frac{d}{dx}max(0.01x,x)=\begin{cases}
\frac{d}{dx}0.01x, &\text{$x<0$} \\ 
\frac{d}{dx}x, &\text{$x\geq0$}
\end{cases}=
\begin{cases}
0.01, &\text{$x<0$} \\ 
1, &\text{$x\geq0$}
\end{cases}$$

```python
torch.nn.LeakyReLU
```

In [ ]:
activation = nn.LeakyReLU(0.01)
input = torch.randn(5)
output = activation(input)
print(f'input: {input}\noutput: {output}')

**Достоинства**: Сохраняет достоинства ReLU, при этом не страдает от проблемы "умирания" 

**Недостатки**: Некоторые исследователи сообщают об успешном применении данной функции активации, но результаты не всегда стабильны.

####  **GELU (Gaussian Error Linear Unit)**

Гауссова ошибка линейного блока. Функция активации, используемая в самых последних трансформерах: Google BERT и OpenAI GPT-2.

$$GELU(x)=xP(X\leq x)=x\Phi(x)=x\cdot \frac{1}{2}[1+erf(\frac{x}{\sqrt{2}})]$$
$$erf(x)=\frac{2}{\sqrt{\pi}}\int_0^xe^{-t^2}dt$$

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/unnamed.png"  width="600">

In [ ]:
activation = nn.GELU()
input = torch.randn(5)*5
output = activation(input)
print(f'input: {input}\noutput: {output}')

**Достоинства**: State-of-the-art функция активации в задачах NLP

####  **Другие модификации ReLU**

* Parametric ReLU (**PReLU**)
Для параметрического ReLU (parametric ReLU, PReLU) угловой коэффициент &alpha; на отрицательном интервале не задается предварительно, а определяется на основе данных. Авторы публикации утверждают, что применение данной функции активации является ключевым фактором, позволившим превзойти уровень человека в задаче распознавания изображений ImageNet. Процесс обратного распространения ошибки и обновления для PReLU достаточно прост и подобен соответствующему процессу для традиционных ReLU.

$$prelu(x)=max(\alpha x,x)$$

* Randomized ReLU (**RReLU**)
Для рандомизированного ReLU (randomized ReLU, RReLU) угловой коэффициент на отрицательном интервале во время обучения генерируется случайным образом из заданного интервала, а во время тестирования остается постоянным. В рамках Kaggle-соревнования National Data Science Bowl (NDSB) RReLU позволили уменьшить переобучение благодаря свойственному им элементу случайности.




* Exponential Linear Unit (**ELU**)
Экспоненциальная линейная единица. Эта функция активации устраняет некоторые проблемы с ReLU и сохраняет некоторые положительные моменты. Для этой функции активации выбирается альфа-значение; общее значение составляет от 0,1 до 0,3.

$$ELU(x)=\begin{cases}
x, &\text{$x>0$} \\ 
\alpha(e^x-1), &\text{$x\leq0$}
\end{cases}$$

Визуализация функций активации:




<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img/more_activations_for_god_of_activations.gif"  width="600">

[Activation Functions Explained](https://mlfromscratch.com/activation-functions-explained/)

#  Обучение нейронной сети

В процессе **обучения** (англ. fit) сеть в определённом порядке просматривает обучающую выборку. Порядок просмотра может быть последовательным и случайным. Некоторые сети, обучающиеся без учителя (например, сети Хопфилда), просматривают выборку только один раз. Другие (например, сети Кохонена), а также сети, обучающиеся с учителем, просматривают выборку множество раз, при этом один полный проход по выборке называется **эпохой обучения** (англ. epoch).

При обучении с учителем набор исходных данных делят на две части — собственно обучающую выборку и тестовые данные, принцип разделения может быть произвольным. Обучающие данные подаются сети для обучения, а проверочные используются для расчета ошибки сети (проверочные данные никогда для обучения сети не применяются). Таким образом, если на проверочных данных ошибка уменьшается, то сеть действительно выполняет обобщение. Если ошибка на обучающих данных продолжает уменьшаться, а ошибка на тестовых данных увеличивается, значит, сеть перестала выполнять обобщение и просто «запоминает» обучающие данные. Это явление называется **переобучением сети** или оверфиттингом (англ. overfit).

##  Прямое распространение

Feedforward neural network - нейронная сеть с прямым распространением сигнала

Рассмотрим **обучение с учителем**, при котором обучаемой сети последовательно представляется набор примеров из обучающего множества. Примеры представляют собой пары эталонных входных воздействий и желаемых выходных сигналов. Процесс обучения проходит циклически, на каждой итерации выполняется расчет сигналов при прямом и обратном распространениях, после чего сигналы ошибок используются для формирования локальных градиентов векторов адаптируемых параметров. Вычисленные локальные градиенты используются для последующей корректировки адаптируемых параметров. 

Используемые режимы обучения – это последовательный режим (online), при котором подстройка параметров происходит после каждого примера, и пакетный (**batch**), при котором подстройка осуществляется на основе кумулятивного локального градиента – суммы локальных градиентов по всем итерациям примеров из обучающего множества. В обоих режимах полный цикл представления множества шаблонов обучения, завершающийся подстройкой параметров, эпохой обучения сети. Для количественной оценки качества работы сети вводится функция потерь.

##  Веса сети

Как и в перцептроне, в нейронной сети используются **веса**. (Иногда используют название синапсы по аналогии с человеческим мозгом). Веса сети - это вещественные числа (чаще от -1 до 1), которых характеризуют влияние входа на выход.

**Нейрон** – базовая единица нейронной сети. У каждого нейрона есть определённое количество входов, куда поступают сигналы, которые суммируются с учётом значимости (веса) каждого входа. Далее сигналы поступают на входы других нейронов. Вес каждого такого «узла» может быть как положительным, так и отрицательным. Например, если у нейрона есть два 'входа', то у него есть и два  весовых значения, которые можно регулировать независимо друг от друга.

###  Как вычислить результат работы нейронной сети

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/12.png"  width="600">

Рассмотрим задачу классификации XOR, то есть на вход подадим 1 и 0, и будем ожидать 1 на выходе. Веса сети определим случайным образом:

$I1=1\quad I2=0$

$w_1=0.45\quad  w_2=0.78\quad 
w_3=-0.12\quad  w_4=0.13$ 

$w_5=1.5\quad  w_6=-2.3$

```python
H1 = I1*W1+I2*W3 = 1*0.45+0*-0.12 = 0.45
H2 = I1*W2+I2*W4 = 1*0.78+0*0.13 = 0.78
```

Для того чтобы значения H1 и H2 не выходили за предельные значения, используется функция активации 

```python
H1_out = sigmoid(H1) = sigmoid(0.45) = 0.61
H2_out = sigmoid(H2) = sigmoid(0.78) = 0.69
```

```python
O1_in = 0.61*1.5+0.69*-2.3=-0.672
O1_out = sigmoid(-0.672)=0.33
```

Ответ нейронной сети O1_out = 0.33, а мы ожидали на выходе 1. О том, как скорректировать веса нужным образом будет рассказано в разделе о методе обратного распространения ошибки.

###  Смещение (bias)

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/13.png"  width="500">

Рассмотрим простой пример. На вход нейрона подаётся вес умноженный на входное значение. После применения функции активации, в зависимости от веса, при всевозможных значениях входа мы можем получить следующие графики:

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img/bias1.png"  width="500">

[A Biased Graph Neural Network Sampler](https://arxiv.org/pdf/2103.01089.pdf)

Но что если мы захотим чтобы при ```x=2``` чтобы сеть выводила ```0```, тогда без веса смещения эту задачу не решить.

Просто изменить крутизну сигмоиды на самом деле не получится - вы хотите иметь возможность сдвинуть всю кривую вправо .

**Смещение** (англ. bias) - это дополнительный коэффициент прибавляющийся ко сумме входов, наличие смещения позволяет сдвинуть функцию активации влево или вправо, что может иметь решающее значение для успешного обучения.

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/14.png"  width="500">

Тогда, при разных смещениях мы можем получить сдвинутые функции активации, что способствует лучшему обучению нейронной сети:

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img/bias3.png"  width="500">

#  Метод обратного распространения ошибки


<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/15.png"  width="800">

Итак, у нас есть все компоненты для обучения 2-х слойной модели:
- предобработали данные
- умножили их на веса
- применили функцию активации
- снова умножили
- нашли значение функции потерь
- нашли градиент
- обновили веса ....
- оценили точность

А как будем искать градиент?
Во второй лекции мы в ручную считали от нее производную. Так как модель поменялась придется делать это заново

Для того что бы упростить этот процесс используется формализм под названием "Алгоритм обратного распространения ошибки" или "Backpropagation"


##  Backpropagation - метод обратного распространения ошибки

**Метод обратного распространения ошибки (англ. backpropagation)** — метод вычисления градиента, который используется при обновлении весов многослойного перцептрона. Метод является модификацией классического метода **градиентного спуска. (англ. gradient descent)** Впервые метод был описан в 1974 г. А. И. Галушкиным, а также независимо и одновременно Полом Дж. Вербосом. Далее существенно развит в 1986 г. Дэвидом И. Румельхартом, Дж. Е. Хинтоном и Рональдом Дж. Вильямсом

Затем в развитии машинного обучения начался период застоя, поскольку компьютеры того времени были не пригодны для создания масштабных моделей. В 2006 году Джеффри Хинтон и Руслан Салахутдинов опубликовали статью, в которой показали, как можно эффективно обучать глубокие нейросети. Но даже тогда они пока не приобрели современный вид.


Первых по-настоящему впечатляющих результатов исследователи искусственного интеллекта достигли в 2012 году, когда почти одновременно появились успешные решения задач распознавания речи и классификации изображений. Тогда же была представлена первая свёрточная нейросеть AlexNet, которая достигла высокой на тот момент точности классификации датасета ImageNet. С тех пор подобные архитектуры довольно широко применяются в разных областях.



##  Идея

Основная идея этого метода состоит в распространении сигналов ошибки от выходов сети к её входам, в направлении, обратном прямому распространению сигналов в обычном режиме работы.

Для возможности применения метода обратного распространения ошибки передаточная функция нейронов должна быть дифференцируема.

Появление алгоритма обратного распространения ошибки вкупе с возрастающими компьютерными мощностиями позволило обучить многослойную нейронную сеть которую можно было применять для решения практических задач.

Архитектура такой сети была разработана в 1989г. Яном Ле Куном. Сеть имела 5 слоев, из них 2 сверточных.

Применялась в США для распознавания рукописных букв на почтовых конвертах до начала 2000г.

Основная идея этого метода состоит в распространении сигналов ошибки от выходов сети к её входам, в направлении, обратном прямому распространению сигналов в обычном режиме работы

## Граф вычислений

Любую нейронную сеть можно представить в виде графа "последовательных действий", где результат вычисляется итеративно, одно действие за другим.

Алгоритм по которому вычисляются веса можно представить в виде графа. Для кода который мы использовали для линейного классификаторо он будет выглядеть так:

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/16.png"  width="700">

И производную от Loss для такого графа можно найти в ручную, что мы и делали.

Однако по мере добавления слоев модель может оказаться намного сложнее. Напримет, так выглядит граф для AlexNet


<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/17.png"  width="800">

Алгоритм обратного распространения ошибки позволяет находить градиенты для любого графа вычислений, если функция которую он описывает дифференцируема.

В его основе лежит правило взятия производной сложной функции (chain rule)

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/gan/L05_7.png"  width="700">

И мы уже пользовались им когда брали производную от CrossEntropyLoss


$$ L =  - \sum_i \log(\dfrac {e^{s_{y_i}}} {\sum_j e^{s_{y_j}}})$$








In [ ]:
def CrossEntropyLoss(self, x, y):
      y = [int(i) for i in y]
      n_features = x.shape[1] # number of features
      n_samples = x.shape[0] # number of samples

      # CalculateCross-entropy loss over a batch 
      scores = np.dot(x, self.W) # logits

      # Softmax
      exp_scores = np.exp(scores - np.max(scores, axis=1, keepdims=True))
      probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

      # Cross Entropy
      correct_logprobs = -np.log(probs[np.arange(n_samples), y])
      loss = np.sum(correct_logprobs) / n_samples

      # Calculate gradient over a batch 
      dW = np.zeros(x.shape)
      # Calculate gradients respect to probs
      probs[np.arange(n_samples), y] -= 1
      probs /= n_samples
      # Use chain rule
      dW = x.T.dot(probs)
      
      return loss, dW

###  Пошаговый разбор метода обратного распространения

Прямой проход (forward):


<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/18.png"  width="600">



Обратный(backward):


<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/19.png"  width="700">


$L = Loss$ - всегда скаляр.

$Loss = L(f(q(x,y),z))$

А что если вход соединен с несколькими вершинами графа, либо у вершины у вершины больше одного выхода?


<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/20.png"  width="500">


В этом случае в месте ветвления можно создать дополнительную вершину, которая будет соответствовать операции копирования.

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/21.png"  width="500">

Тогда при обратном распространении входящим (upstream) будет вектор значений. 

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/22.png"  width="700">

Таким образом можно выделить шаблоны для получения градиентов при базовых операциях:

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/gan/L05_8_5.png"  width="700">

А общее правило взятия градиентов можно представить следующим образом:

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img/b0.PNG"  width="600">

В качестве более сложного примера, рассмотрим следующую функцию:

$$f(x,w)=\frac{1}{1+e^{-(w_0x_0+w_1x_1+w_2)}}$$

Представим ее в виде графа вычислений:

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/24.png"  width="700">

Нам нужно найти градиенты $\frac{df}{dw_0}$, $\frac{df}{dw_1}$, $\frac{df}{dw_2}$, $\frac{df}{dx_0}$ и $\frac{df}{dx_1}$. Пусть $w_0=2,\;x_0=-1,\;w_1=-3,\;x_1=-2,\;w_2=-3$. Для данного примера сделаем прямой проход через граф вычислений, на выходе получим $f=0.73$:

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/25.png"  width="800">

Далее в соостветствии с алгоритмом обратного распространения ошибки, расчитаем градиенты, проходясь последовательно по графу вычислений:

Для начала посчитаем производную функции $\frac{df}{df}$, которая будет равна единице. Движемся дальше &mdash; следующая вершина содержит функцию $f(x)=\frac{1}{x}$, её производная $\frac{df}{dx}=-\frac{1}{x^2}$

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/26.png"  width="800">

В следующем узле находится функция $f(x)=1+x$. $+1$ &mdash; константа, производная которой равна нулю, производная всего выражения $\frac{df}{dx}=1$:

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/27.png"  width="800">

Третья вершина — это экспонента $f(x)=e^x$, её производная $\frac{df}{dx}=e^x$

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/28.png"  width="800">

Четвертая вершина — это умножение на константу $f(x)=ax$, её производная $\frac{df}{dx}=a$

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/29.png"  width="800">

Итак, на каждом шаге мы умножаем локальный градиент на восходящий, чтобы выполнялось цепное правило дифференцирования.

Мы подошли к узлу сложения с двумя входящими в него рёбрами, для которого известен восходящий градиент, равный $0.2$. В вершине сложения локальный градиент по отношению к каждому из входов будет равен $1$. Поэтому мы просто берём единицу (локальный градиент) и умножаем её на восходящий градиент для обоих рёбер. Аналогично поступаем и со следующим узлом сложения. 

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/30.png"  width="800">

Далее перед ними находится вершина умножения. В этом случае локальный градиент по отношению к каждому из входов просто является значением другого входа, умноженным на восходящий градиент. Аналогично поступаем со второй вершиной напротив $w_1$ и $x_1$

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/31.png"  width="800">

В данном примере мы разбивали граф вычислений на множество простейших вершин. Но на самом деле это не всегда необходимо: вершины можно группировать друг с другом, если они образуют дифференцируемую функцию. Это может существенно сократить и упростить вычисления.

В частности, можно заметить, что в данном примере выражение $w_0x_0+w_1x_1+w_2$ подается на вход в функцию сигмоиды, мы можем взять ту часть графа, которая соответствует сигмоиде, и объединить несколько узлов в один. У сигмоиды есть очень полезное свойство: её производная легко выражается через саму функцию:

$$\frac{d\sigma{(x)}}{dx}=(1-\sigma{(x)})\sigma{(x)}$$

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/32.png"  width="800">

В коде, без использования библиотек подсчёт градиентов можно записать как:

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/33.png"  width="800">

Объектно-ориентированный код:

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/34.png"  width="700">

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/gan/
L05_9_2.png"  width="700">

Pytorch

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/35.png"  width="700">

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img_licence/l5_1.png"  width="800">

###  Обратное распространение для векторов:

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/gan/L05_10.png"  width="700">

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/gan/L05_10_1.png"  width="700">

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/gan/L05_10_2.png"  width="700">

###  Анимация работы метода обратного распространения ошибки

<img src ="http://edunet.kea.su/repo/src/L05_Neural_networks/img/backprop_animation.gif"  width="500">

###  Backprop in PyTorch


Рассмотрим пример реализации шага обратного прохода на примере вычисления квадрата ошибки для линейной регрессии (для простоты не будем рассматривать смещение):

$$y=w\cdot x, \quad при \;x=[1,2,3,4],\;y=[2,4,6,8],\;w=1$$

В данном примере видно, что предсказанный моделью $\hat{y}=[1,2,3,4]$ не совпадает с истинными значениями $y$, и соотвтественно квадратичная ошибка для такого примера будет $$MSE=\frac{1}{4}\sum_{i=1}^4E_i^2=\frac{1}{4}\sum_{i=1}^4(\hat{y}_i-y_i)^2=\frac{1+4+9+16}{4}=7.5$$

Градиент весов $w$ вычисляется следующим образом в соответсвии с chain rule:

$$\frac{d MSE}{d w} = \frac{\partial MSE}{\partial E}\cdot \frac{\partial E}{\partial \hat{y}}\cdot \frac{\partial \hat{y}}{\partial w}$$

Рассчитаем его с использованием PyTorch:



In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import torch

X = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
Y = torch.tensor([2, 4, 6, 8], dtype=torch.float32)

# This is the parameter we want to optimize -> requires_grad=True
W = torch.tensor(1.0, dtype=torch.float32, requires_grad=True)

# forward pass to compute MSE
Y_hat = W*X
E = Y_hat-Y
SE = E**2
MSE = SE.mean()
print(f"MSE = {MSE}")

# backward pass to compute gradient dMSE/dw
MSE.backward()
print (f"W.grad = {W.grad}")
print (f"E.grad = {E.grad}")


В данном примере мы произвели следующие рассчеты:

$\frac{\partial MSE}{\partial E}=\frac{\sum\partial E^2}{\partial E}=\frac{1}{4}\cdot2\cdot E=\frac{1}{2}*[-1, -2, -3, -4]=[-0.5, -1, -1.5, -2]\quad *-поэлементное\;умножение$

$\frac{\partial E}{\partial \hat{y}}=\frac{\partial (\hat{y}-y)}{\partial \hat{y}}=1$

$\frac{\partial \hat{y}}{\partial w}=\frac{\partial wx}{\partial w}=x=[1, 2, 3, 4]$

$\frac{d MSE}{d w} = \frac{\partial MSE}{\partial E}\cdot \frac{\partial E}{\partial \hat{y}}\cdot \frac{\partial \hat{y}}{\partial w}=\sum[-0.5, -1, -1.5, -2]*[1, 2, 3, 4]=-0.5-2-4.5-8=-15$

`MSE.backward()` автоматически вычисляет градиент $\frac{dMSE}{dw}$ при указании `requires_grad=True`. 
Результаты вычислений будут храниться в `W.grad`. Для всех промежуточных переменных градиенты не сохраняются, поэтому попытка обратиться, например, к `E.grad` выдает `None`. 

Также после однократного обратного прохода, в целях экономии памяти, граф, используемый для вычисления градиента будет удаляться и следующий запуск `MSE.backward()` будет выдавать ошибку:

In [ ]:
MSE.backward() # Error on second backward call

Чтобы сохранить вычислительный граф, для аргумента `retain_graph` функции `bacward()` нужно указать значение `True`. Также может быть полезным сохранять значения градиентов для промежуточных переменных, это делается с помощью функции `tensor.retain_grad()`. В таком случае, значения градиентов, полученные на следующих итерациях обратного распространения ошибки, будут складываться с текущими значениями градиентов.

Градиенты переменных, для которых был указан `requires_grad=True` сохраняются автоматически, чтобы избежать их накопления при многократном итерировании алгоритма обратного распространения, нужно обнулять градиент на каждом шаге с помошью функции `tensor.grad.zero_()`

In [ ]:
X = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
Y = torch.tensor([2, 4, 6, 8], dtype=torch.float32)

# This is the parameter we want to optimize -> requires_grad=True
W = torch.tensor(1.0, dtype=torch.float32, requires_grad=True)

# forward pass to compute MSE
Y_hat = W * X
E = Y_hat - Y
E.retain_grad() # Save grads for intermediate tensor E in memory
SE = E**2
MSE = SE.sum().div(4)

print("========== Backprop 1 ==============")
MSE.backward(retain_graph=True)
print (f"dMSE/dE = {E.grad}")
print (f"dMSE/dW = {W.grad}")

print("========== Backprop 2 ==============")
MSE.backward(retain_graph=True)
# Gradients are accumulated
print (f"dMSE/dE = {E.grad}")
print (f"dMSE/dW = {W.grad}")

print("========== Backprop 3 ==============")
W.grad.zero_() # Nullify gradients for W for the next iteration
MSE.backward(retain_graph=True)
# Gradients for W are not accumulated, but not for E
print (f"dMSE/dE = {E.grad}")
print (f"dMSE/dW = {W.grad}")

Итак, мы умеем вычислять градиент $\frac{\partial MSE}{\partial w}$ для нашего примера. Теперь давайте с его помощью оптимизируем веса, используя алгоритм обратного распростронения ошибки:

In [ ]:
X = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
Y = torch.tensor([2, 4, 6, 8], dtype=torch.float32)

W = torch.tensor(1.0, dtype=torch.float32, requires_grad=True)

# Define model output
def forward(X):
    return W*X

# Compute MSE loss
def loss(Y_hat, Y):
    return ((Y_hat-Y)**2).mean()

print(f'Prediction before training: f(X) = {forward(X)}')

# Training
learning_rate = 0.005
n_iters = 101

for epoch in range(n_iters):
    # Propagate forward
    Y_hat = forward(X)

    # Compute loss
    MSE = loss(Y_hat, Y)

    # Propagate backward, compute gradients
    MSE.backward()

    # Update weights
    with torch.no_grad(): # We don't want this step to be the part of the computational graph
        W -= learning_rate*W.grad 
    
    # Nullify gradients after updating to avoid their accumulation
    W.grad.zero_()

    if epoch % 10 == 0:
        print(f'epoch {epoch}: w = {W.item():.3f}, loss = {MSE.item():.8f}')

print(f'Prediction after training: f(X) = {forward(X)}')

Видно, что наш подход позволяет оптимизировать вес $w$ регрессии из примера и таким образом добиться почти идеального предсказания нашей модели, однако в данном подходе дополнительно можно автоматизировать вычисление функции потерь и обновление параметров с учетом градиента, используя готовые функции потерь из `torch.nn` и оптимизаторы из `torch.optim`.

In [ ]:
import torch.nn as nn

X = torch.tensor([1, 2, 3, 4], dtype=torch.float32)
Y = torch.tensor([2, 4, 6, 8], dtype=torch.float32)

W = torch.tensor(1.0, dtype=torch.float32, requires_grad=True)

# Define model output
def forward(X):
    return W*X

print(f'Prediction before training: f(X) = {forward(X)}')

# Training
learning_rate = 0.005
n_iters = 101

# Define loss and optimizer
loss = nn.MSELoss()
optimizer = torch.optim.SGD([W], lr=learning_rate)

for epoch in range(n_iters):
    # Propagate forward
    Y_hat = forward(X)

    # Compute loss
    MSE = loss(Y, Y_hat)

    # Propagate backward, compute gradients
    MSE.backward()

    # Update weights
    optimizer.step()

    # Nullify the gradients after updating
    optimizer.zero_grad()

    if epoch % 10 == 0:
        print(f'epoch {epoch}: W = {W.item():.3f} loss = {MSE.item():.8f}')

print(f'Prediction after training: f(X) = {forward(X)}')

##  Преимущества и недостатки

Несмотря на многочисленные успешные применения метода обратного распространения, он не является универсальным решением. Больше всего неприятностей приносит неопределённо долгий процесс обучения. В сложных задачах для обучения сети могут потребоваться дни или даже недели, она может и вообще не обучиться. Причиной может быть одна из описанных ниже.

* Паралич сети

    В процессе обучения сети значения весов могут в результате коррекции стать очень большими величинами. Это может привести к тому, что все или большинство нейронов будут функционировать при очень больших значениях, в области, где производная функции очень мала. Так как посылаемая обратно в процессе обучения ошибка пропорциональна этой производной, то процесс обучения может практически замереть. 
    
    В теоретическом отношении эта проблема плохо изучена. Обычно этого избегают уменьшением размера шага, но это увеличивает время обучения. Различные эвристики использовались для предохранения от паралича или для восстановления после него, но пока что они могут рассматриваться лишь как экспериментальные.


* Локальные минимумы

    Метод градиентного спуска может застрять в локальном минимуме, так и не попав в глобальный минимум.

    Обратное распространение использует разновидность градиентного спуска, то есть осуществляет спуск вниз по поверхности ошибки, непрерывно подстраивая веса в направлении к минимуму. 
    
    Поверхность ошибки сложной сети сильно изрезана и состоит из холмов, долин, складок и оврагов в пространстве высокой размерности. Сеть может попасть в локальный минимум (неглубокую долину), когда рядом имеется гораздо более глубокий минимум. В точке локального минимума все направления ведут вверх, и сеть не способна из него выбраться. Основную трудность при обучении нейронных сетей составляют как раз методы выхода из локальных минимумов: каждый раз выходя из локального минимума снова ищется следующий локальный минимум тем же методом обратного распространения ошибки до тех пор, пока найти из него выход уже не удаётся.

* Размер шага

    Если размер шага фиксирован и очень мал, то сходимость слишком медленная, если же он фиксирован и слишком велик, то может возникнуть паралич или постоянная неустойчивость. Эффективно увеличивать шаг до тех пор, пока не прекратится улучшение оценки в данном направлении антиградиента и уменьшать, если такого улучшения не происходит. П. Д. Вассерман описал адаптивный алгоритм выбора шага, автоматически корректирующий размер шага в процессе обучения. В книге А. Н. Горбаня предложена разветвлённая технология оптимизации обучения.

* Переобучение

    Следует также отметить возможность переобучения сети (overfitting), что является скорее результатом ошибочного проектирования её топологии и/или неправильным выбором критерия остановки обучения. При переобучении теряется свойство сети обобщать информацию. Весь набор образов, предоставленных к обучению, будет выучен сетью, но любые другие образы, даже очень похожие, могут быть распознаны неверно.

#  Пример простой сети на датасете mnist

В PyTorch для создания нейронных сетей требуется отнаследоваться от класса nn.Module и переопределить метод forward, в который подаются входные данные, и ожидаются выходные данные.

[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module)

В класс мы добавляем две переменные, два слоя Linear. Linear -это слой, позволяющий, умножить веса на входной вектор и добавить смещение. Первый параметр - размер входного вектора, второй - размер выходного.

В методе forward мы указываем последовательность применения операций для получения результата. Сначала изменим представление входного вектора, чтобы от изменения batch_size у нас ничего не сломалось.

Далее идёт первый слой, после него функция активации relu и второй слой, возвращающий вектор длиной 10, означающий принадлежность к одному из классов.

In [ ]:
import torch
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x): # Called inside __call__ method
        x = x.view(-1, 28*28)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

Наследование от nn.Module позволяет объединять блоки:

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)

Загрузим датасет

In [ ]:
import torchvision
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5), (0.5))])

trainset = MNIST(root='./MNIST', train=True, download=True, transform=transform)
testset = MNIST(root='./MNIST', train=False, download=True, transform=transform)

batch_size = 64
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

In [ ]:
net = Net()

Определим нашу лосс-функцию

In [ ]:
loss_function = nn.CrossEntropyLoss()

Определим оптимайзер для найшей сети. Оптимайзер, в библиотеках для нейронных сетей -- это сущность, осуществляющая градиентный спуск. Подробнее об оптимайзере будет рассказано в дальнейших лекциях.

In [ ]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.001)

Обучим сеть десять эпох

In [ ]:
import torch.nn.functional as F

epochs = 10
loss_hist = []
for ep in range(epochs):
    hist_loss = 0
    for _, data in enumerate(trainloader, 0): # get bacth
        # parse batch
        images, labels = data
        # sets the gradients of all optimized tensors to zero.
        optimizer.zero_grad() 
        # get outputs
        outputs = net(images) 
        # calculate loss
        loss = loss_function(outputs, labels)
        # calculate gradients
        loss.backward() 
        # performs a single optimization step (parameter update).
        optimizer.step()
        hist_loss += loss.item()
    loss_hist.append(hist_loss /len(trainloader))
    print(f"Epoch={ep} loss={loss_hist[ep]:.4f}")

In [ ]:
import matplotlib.pyplot as plt
plt.plot(range(epochs), loss_hist)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

Давайте посчитаем `accuracy`:

In [ ]:
def calaculate_accuracy(model, dataloader):
    correct, total = 0, 0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            outputs = model.forward(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

In [ ]:
acc_train = calaculate_accuracy(net, trainloader)
print(f"Accuracy train= {acc_train}")
acc_test = calaculate_accuracy(net, testloader)
print(f"Accuracy test= {acc_test}")

In [ ]:
images, classes = next(iter(testloader))
images.shape

In [ ]:
outputs = net.forward(images)
outputs.shape

In [ ]:
images = torch.reshape(images, (64, 28, 28))
images.shape

In [ ]:
images = images[:10]
images.shape

In [ ]:
import numpy as np

outputs = outputs[:10]
outputs.shape
digits = np.argmax(outputs.detach().numpy(), axis=1)

In [ ]:
for digit, image in zip(digits, images):
    print(digit)
    pixels = image.reshape((28, 28))
    plt.imshow(pixels, cmap='gray')
    plt.show()

# Ссылки:


[StatSoft. Радиальная базисная функция](http://statsoft.ru/home/textbook/modules/stneunet.html#radial)

[Важность функции потери в машинном обучении](https://www.machinelearningmastery.ru/importance-of-loss-function-in-machine-learning-eddaaec69519/)

[Understanding Categorical Cross-Entropy Loss, Binary Cross-Entropy Loss, Softmax Loss, Logistic Loss, Focal Loss and all those confusing names](https://gombru.github.io/2018/05/23/cross_entropy_loss/)

[Функции активации нейросети: сигмоида, линейная, ступенчатая, ReLu, tahn](https://neurohive.io/ru/osnovy-data-science/activation-functions/)

[Объясненные современные функции активации: GELU, SELU, ELU, ReLU и другие](https://www.machinelearningmastery.ru/state-of-the-art-activation-functions-explained-gelu-selu-elu-relu-and-more-a4247171ca4e/)